In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import logging
import sys

import pandas as pd
import plotly.express as px
import sqlalchemy as sa

import energy_comms
import pudl

In [3]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Summary Stats
Note: This isn't a dynamically updating table

|    | Mines | Gens | Combined |
|----| ----- | ---- | -------- |
num qualifying records | 5143 | 4470 | 9600 |
num lat, lon pairs| 4737  | 319 | 5056 |
num Census tracts with closures| 966 | 309 | 1236 |
num Census tracts total (with adjacent tracts) | 2999 | 2486 | 4992 |
num counties with closures | 322 | 275 | 533 | 
num counties total (with adjacent counties) | 840 | 1297 | 1631 |

In [4]:
# get basic stats about mine or generators dataframe
def get_df_stats(df, census_geom):
    n_lat_lon = len(df[["latitude", "longitude"]].drop_duplicates())
    n_primary_geoms = len(df[f"{census_geom}_id_fips"].drop_duplicates())
    n_all_geoms = len(
        pd.concat([df[f"{census_geom}_id_fips"], df.adjacent_id_fips.explode()]).drop_duplicates())
    
    return n_lat_lon, n_primary_geoms, n_all_geoms

# MSHA Mines Data

In [5]:
from energy_comms.extract.msha import extract as msha_extract
from energy_comms.transform.msha import transform as msha_transform

### Start with looking at qualifying Census tracts, as specified by criteria

In [156]:
# add census tract data onto msha closed mines
msha_df = msha_transform(msha_extract())

Finding intersecting Census geometries.
Finding adjacent Census geometries.


In [7]:
n_lat_lon, n_primary_tracts, n_all_tracts = get_df_stats(msha_df, census_geom="tract")

print(f"Number of qualifying mines: {len(msha_df)}")
print(f"Number of unique lat, lon pairs: {n_lat_lon}")
print(f"Number of Census tracts with closed mines: {n_primary_tracts}")
print(f"Number of qualifying Census tracts (including adjacent tracts): {n_all_tracts}")

Number of qualifying mines: 5463
Number of unique lat, lon pairs: 5031
Number of Census tracts with closed mines: 986
Number of qualifying Census tracts (including adjacent tracts): 3067


In [8]:
cols = msha_df.columns.to_list()
msha_df[cols[:2] + cols[-3:] + cols[2:-3]]

,mine_id,current_mine_name,tract_id_fips,tract_name_census,adjacent_id_fips,coal_metal_ind,current_mine_type,current_mine_status,current_status_dt,current_controller_id,current_controller_name,current_operator_id,current_operator_name,state,bom_state_cd,fips_cnty_cd,fips_cnty_nm,cong_dist_cd,company_type,current_controller_begin_dt,district,office_cd,office_name,assess_ctrl_no,primary_sic_cd,primary_sic,primary_sic_cd_1,primary_sic_cd_sfx,secondary_sic_cd,secondary_sic,secondary_sic_cd_1,secondary_sic_cd_sfx,primary_canvass_cd,primary_canvass,secondary_canvass_cd,secondary_canvass,current_103i,current_103i_dt,portable_operation,portable_fips_st_cd,days_per_week,hours_per_shift,prod_shifts_per_day,maint_shifts_per_day,no_employees,part48_training,longitude,latitude,avg_mine_height,mine_gas_category_cd,methane_liberation,no_producing_pits,no_nonproducing_pits,no_tailing_ponds,pillar_recovery_used,highwall_miner_used,multiple_pits,miners_rep_ind,safety_committee_ind,miles_from_office,directions_to_mine,nearest_town,geometry,index_right
63,100163,Arkadelphia,01043965700,Census Tract 9657,"[01127021900, 01009050601, 01127020800, 010439...",C,Surface,Abandoned,2006-02-03,0125021,Dewayne Kennedy; Autumn Kennedy,0145013,Southern Mineral Resources,AL,1,43,Cullman,4.0,Other,01/02/2020,M3,M3661,Birmingham AL Field Office (A),000063009,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,10/01/1981,N,NaN,0,0.0,0.0,0.0,0.0,N,-86.971944,33.910555,24.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,"16 miles north of Jasper, AL on Hwy 69N, then ...","Arkadelphia, Alabama",POINT (-86.97194 33.91056),1178.0
86,100323,Chetopa,01073011600,Census Tract 116,"[01073012001, 01073012103, 01073012104, 010730...",C,Surface,Abandoned,2001-09-20,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,73,Jefferson,7.0,Corporation,04/21/1978,M3,M3661,Birmingham AL Field Office (A),010032303793,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,01/01/1978,N,NaN,0,NaN,NaN,1.0,0.0,N,-86.948300,33.668300,77.0,NaN,NaN,0.0,1.0,NaN,N,N,N,N,Y,0,NaN,Graysville,POINT (-86.94830 33.66830),201.0
91,100328,Bessie Mine,01073012104,Census Tract 121.04,"[01073012302, 01073012403, 01073012200, 010730...",C,Facility,Abandoned and Sealed,2012-11-05,C00992,Walter Energy Incorporated,P01155,Jim Walter Resources Inc,AL,1,73,Jefferson,7.0,Other,10/26/1994,M3,M3661,Birmingham AL Field Office (A),000119387,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,12/01/1989,N,NaN,0,0.0,0.0,0.0,0.0,N,-87.035555,33.636111,96.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,NaN,Graysville,POINT (-87.03556 33.63611),196.0
97,100340,Gorgas #7 Preparation Plant,01127021400,Census Tract 214,"[01127021600, 01127021300, 01127021500, 011270...",C,Facility,Abandoned,2002-03-28,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,127,Walker,4.0,Corporation,04/25/1978,M3,M3661,Birmingham AL Field Office (A),010034003685,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,04/07/1986,N,NaN,0,0.0,1.0,1.0,NaN,N,-87.265556,33.646389,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,Y,0,NaN,Goodsprings,POINT (-87.26556 33.64639),43.0
116,100406,Beltona Mine,01073011400,Census Tract 114,"[01073011600, 01073011500, 01073011704, 011270...",C,Surface,Abandoned,2009-06-30,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,73,Jefferson,7.0,Corporation,12/07/1973,M3,M3661,Birmingham AL Field Office (A),010040603515,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,10/01/1981,N,NaN,0,0.0,0.0,0.0,NaN,N,-86.853888,33.802777,27.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,NaN,Warrior,POINT (-86.85389 33.80278),203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Visualize qualifying census tracts

Note: Everything runs very slow if the full plot with all Census tracts is rendered in the notebook, so I don't render it here. 


The cell below it allows you to save the full map as an HTML, which seems to work better.


For the sake of visualization in this notebook, I created a non-interactive matplotlib map.

In [11]:
from energy_comms.output.basic_viz import create_geometries_df, make_plotly_map, make_matplotlib_map

In [12]:
msha_tract_geoms = create_geometries_df(msha_df)

In [ ]:
make_plotly_map(msha_tract_geoms, "msha_tracts.html")

In [ ]:
make_matplotlib_map(msha_tract_geoms)

### What about using county instead of tract?

In [13]:
msha_county_df = msha_transform(msha_extract(), census_geometry="county")

Finding intersecting Census geometries.
Finding adjacent Census geometries.


In [14]:
n_lat_lon, n_primary_counties, n_all_counties = get_df_stats(msha_county_df, census_geom="county")

print(f"Number of qualifying mines: {len(msha_county_df)}")
print(f"Number of unique lat, lon pairs: {n_lat_lon}")
print(f"Number of Census tracts with closed mines: {n_primary_counties}")
print(f"Number of qualifying Census tracts (including adjacent tracts): {n_all_counties}")

Number of qualifying mines: 5463
Number of unique lat, lon pairs: 5031
Number of Census tracts with closed mines: 329
Number of qualifying Census tracts (including adjacent tracts): 861


In [15]:
cols = msha_county_df.columns.to_list()
msha_county_df[cols[:2] + cols[-3:] + cols[2:-3]]

,mine_id,current_mine_name,county_id_fips,county_name_census,adjacent_id_fips,coal_metal_ind,current_mine_type,current_mine_status,current_status_dt,current_controller_id,current_controller_name,current_operator_id,current_operator_name,state,bom_state_cd,fips_cnty_cd,fips_cnty_nm,cong_dist_cd,company_type,current_controller_begin_dt,district,office_cd,office_name,assess_ctrl_no,primary_sic_cd,primary_sic,primary_sic_cd_1,primary_sic_cd_sfx,secondary_sic_cd,secondary_sic,secondary_sic_cd_1,secondary_sic_cd_sfx,primary_canvass_cd,primary_canvass,secondary_canvass_cd,secondary_canvass,current_103i,current_103i_dt,portable_operation,portable_fips_st_cd,days_per_week,hours_per_shift,prod_shifts_per_day,maint_shifts_per_day,no_employees,part48_training,longitude,latitude,avg_mine_height,mine_gas_category_cd,methane_liberation,no_producing_pits,no_nonproducing_pits,no_tailing_ponds,pillar_recovery_used,highwall_miner_used,multiple_pits,miners_rep_ind,safety_committee_ind,miles_from_office,directions_to_mine,nearest_town,geometry,index_right
63,100163,Arkadelphia,01043,Cullman County,"[01127, 01009, 01133, 01095, 01103, 01079]",C,Surface,Abandoned,2006-02-03,0125021,Dewayne Kennedy; Autumn Kennedy,0145013,Southern Mineral Resources,AL,1,43,Cullman,4.0,Other,01/02/2020,M3,M3661,Birmingham AL Field Office (A),000063009,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,10/01/1981,N,NaN,0,0.0,0.0,0.0,0.0,N,-86.971944,33.910555,24.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,"16 miles north of Jasper, AL on Hwy 69N, then ...","Arkadelphia, Alabama",POINT (-86.97194 33.91056),2251.0
86,100323,Chetopa,01073,Jefferson County,"[01007, 01117, 01125, 01115, 01127, 01009]",C,Surface,Abandoned,2001-09-20,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,73,Jefferson,7.0,Corporation,04/21/1978,M3,M3661,Birmingham AL Field Office (A),010032303793,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,01/01/1978,N,NaN,0,NaN,NaN,1.0,0.0,N,-86.948300,33.668300,77.0,NaN,NaN,0.0,1.0,NaN,N,N,N,N,Y,0,NaN,Graysville,POINT (-86.94830 33.66830),2140.0
91,100328,Bessie Mine,01073,Jefferson County,"[01007, 01117, 01125, 01115, 01127, 01009]",C,Facility,Abandoned and Sealed,2012-11-05,C00992,Walter Energy Incorporated,P01155,Jim Walter Resources Inc,AL,1,73,Jefferson,7.0,Other,10/26/1994,M3,M3661,Birmingham AL Field Office (A),000119387,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,12/01/1989,N,NaN,0,0.0,0.0,0.0,0.0,N,-87.035555,33.636111,96.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,NaN,Graysville,POINT (-87.03556 33.63611),2140.0
97,100340,Gorgas #7 Preparation Plant,01127,Walker County,"[01125, 01073, 01057, 01009, 01043, 01093, 01133]",C,Facility,Abandoned,2002-03-28,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,127,Walker,4.0,Corporation,04/25/1978,M3,M3661,Birmingham AL Field Office (A),010034003685,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,04/07/1986,N,NaN,0,0.0,1.0,1.0,NaN,N,-87.265556,33.646389,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,Y,0,NaN,Goodsprings,POINT (-87.26556 33.64639),2159.0
116,100406,Beltona Mine,01073,Jefferson County,"[01007, 01117, 01125, 01115, 01127, 01009]",C,Surface,Abandoned,2009-06-30,C00866,Drummond Company Inc,P17353,"Drummond Company, Inc.",AL,1,73,Jefferson,7.0,Corporation,12/07/1973,M3,M3661,Birmingham AL Field Office (A),010040603515,122200.0,Coal (Bituminous),1222.0,0.0,NaN,NaN,NaN,NaN,2.0,Coal,NaN,NaN,Removed From 103I Status,10/01/1981,N,NaN,0,0.0,0.0,0.0,NaN,N,-86.853888,33.802777,27.0,NaN,NaN,NaN,0.0,NaN,N,N,N,N,Y,0,NaN,Warrior,POINT (-86.85389 33.80278),2140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88039,480

### Visualize qualifying counties

In [16]:
msha_county_geoms = create_geometries_df(msha_county_df, census_geometry="county")

In [ ]:
make_plotly_map(msha_county_geoms, "msha_counties.html")

In [ ]:
make_matplotlib_map(msha_county_geoms)

# EIA 860m Data

In [9]:
from energy_comms.extract.eia860 import extract as eia_extract
from energy_comms.transform.eia860 import transform as eia_transform

### Start with looking at Census tract level, as specified by criteria

In [10]:
raw_gens = eia_extract()

Started with 81265 missing BA Codes out of 185357 records (43.84%)
Ended with 14923 missing BA Codes out of 185357 records (8.05%)
91.6% of plant records have consistently reported BA Codes
Before any filling treatment has been applied. 43.8% of records have no BA codes
Backfilling and consistent value is the same. Filled w/ most consistent BA code. 10.9% of records have no BA codes
SWPP is most consistent value. Filled w/ oldest BA code. 8.3% of records have no BA codes
NWMT is most consistent value. Filled w/ oldest BA code. 8.3% of records have no BA codes
Two or more years of oldest BA code. Filled w/ oldest BA code. 8.1% of records have no BA codes
Filling technology type
Filled technology_type coverage now at 98.3%


In [22]:
eia_df = eia_transform(raw_gens)

Finding intersecting Census geometries.
Finding adjacent Census geometries.


In [23]:
n_lat_lon, n_primary_tracts, n_all_tracts = get_df_stats(eia_df, census_geom="tract")

print(f"Number of qualifying generators: {len(eia_df)}")
print(f"Number of unique lat, lon pairs: {n_lat_lon}")
print(f"Number of Census tracts with closed generators: {n_primary_tracts}")
print(f"Number of qualifying Census tracts (including adjacent tracts): {n_all_tracts}")

Number of qualifying generators: 744
Number of unique lat, lon pairs: 319
Number of Census tracts with closed generators: 309
Number of qualifying Census tracts (including adjacent tracts): 2486


In [24]:
cols = eia_df.columns.to_list()
eia_df[cols[:2] + cols[-3:] + cols[2:-3]]

,report_date,plant_id_eia,tract_id_fips,tract_name_census,adjacent_id_fips,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,bga_source,bypass_heat_recovery,capacity_mw,carbon_capture,city,cofire_fuels,county,current_planned_operating_date,data_maturity,deliver_power_transgrid,distributed_generation,duct_burners,energy_source_1_transport_1,energy_source_1_transport_2,energy_source_1_transport_3,energy_source_2_transport_1,energy_source_2_transport_2,energy_source_2_transport_3,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,energy_storage_capacity_mwh,ferc_qualifying_facility,fluidized_bed_tech,fuel_type_code_pudl,fuel_type_count,latitude,longitude,minimum_load_mw,multiple_fuels,nameplate_power_factor,net_capacity_mwdc,operating_date,operating_switch,operational_status,operational_status_code,original_planned_operating_date,other_combustion_tech,other_modifications_date,other_planned_modifications,owned_by_non_utility,ownership_code,planned_derate_date,planned_energy_source_code_1,planned_modifications,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_derate_mw,planned_net_winter_capacity_uprate_mw,planned_new_capacity_mw,planned_new_prime_mover_code,planned_repower_date,planned_retirement_date,planned_uprate_date,previously_canceled,prime_mover_code,pulverized_coal_tech,reactive_power_output_mvar,retirement_date,rto_iso_lmp_node_id,rto_iso_location_wholesale_reporting_id,solid_fuel_gasification,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,state,stoker_tech,street_address,subcritical_tech,summer_capacity_estimate,summer_capacity_mw,summer_estimated_capability_mw,supercritical_tech,switch_oil_gas,syncronized_transmission_grid,technology_description,time_cold_shutdown_full_load_code,timezone,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,unit_id_pudl,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,geometry,index_right
706,2022-01-01,65285,24033800900,Census Tract 8009,"[24009860701, 24009860600, 24017851400, 240178...",1811,Chalk Point Steam,12653,1869,Mirant Mid-Atlantic LLC,ST2,False,<NA>,False,364.0,<NA>,Aquasco,<NA>,Prince Georges,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BIT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,coal,2,38.544400,-76.686100,NaN,<NA>,NaN,NaN,1965-03-01,<NA>,retired,RE,NaT,<NA>,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,ST,True,NaN,2021-06-01,50816,chalkpt 20 kv chlkg2,False,<NA>,<NA>,<NA>,<NA>,MD,<NA>,P.O. 10,<NA>,<NA>,336.9,NaN,True,<NA>,<NA>,Conventional Steam Coal,<NA>,America/New_York,X,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,344.5,NaN,20608,POINT (-76.68610 38.54440),30261
27798,2022-01-01,1771,26041970900,Census Tract 9709,"[26041990000, 26041970700, 26041971100, 260419...",1849,Escanaba,58246,1658,Escanaba Operating Services LLC,2,False,<NA>,False,11.5,<NA>,Escanaba,<NA>,Delta,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BIT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,coal,2,45.771667,-87.063920,NaN,<NA>,NaN,NaN,1958-05-01,<NA>,retired,RE,NaT,<NA>,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,ST,<NA>,NaN,2016-01-01,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,MI,True,2000 Power Plant Rd,True,<NA>,13.0,NaN,<NA>,<NA>,<NA>,Conventional Steam Coal,<NA>,America/Detroit,X,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,13.0,NaN,49829,POINT (-87.06392 45.77167),34189
27800,2022-01-01,1769,26103000600,Census Tract 6,"[26103000200, 26103000500, 26103000700, 261030...",466,Presque Isle,20847,363,Wisconsin Electric Power Co,9,False,<NA>,False,90.0,<NA>,Marquette,<NA>,Marquette,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,SUB,<NA>,<NA>,<NA>,<NA>,<NA

In [25]:
eia_tract_geoms = create_geometries_df(eia_df, census_geometry="tract")

In [ ]:
make_matplotlib_map(eia_tract_geoms)

There is a closed generator in Hawaii but here's an only CONUS visual.

In [ ]:
make_matplotlib_map(eia_tract_geoms, only_conus=True)

### What about using county instead of Census tract?

In [26]:
eia_county_df = eia_transform(raw_gens, census_geometry="county")

Finding intersecting Census geometries.
Finding adjacent Census geometries.


In [27]:
n_lat_lon, n_primary_counties, n_all_counties = get_df_stats(eia_county_df, census_geom="county")

print(f"Number of qualifying generators: {len(eia_county_df)}")
print(f"Number of unique lat, lon pairs: {n_lat_lon}")
print(f"Number of Census counties with closed generators: {n_primary_counties}")
print(f"Number of qualifying Census counties (including adjacent counties): {n_all_counties}")

Number of qualifying generators: 744
Number of unique lat, lon pairs: 319
Number of Census counties with closed generators: 275
Number of qualifying Census counties (including adjacent counties): 1297


In [28]:
cols = eia_county_df.columns.to_list()
eia_county_df[cols[:2] + cols[-3:] + cols[2:-3]]

,report_date,plant_id_eia,county_id_fips,county_name_census,adjacent_id_fips,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,bga_source,bypass_heat_recovery,capacity_mw,carbon_capture,city,cofire_fuels,county,current_planned_operating_date,data_maturity,deliver_power_transgrid,distributed_generation,duct_burners,energy_source_1_transport_1,energy_source_1_transport_2,energy_source_1_transport_3,energy_source_2_transport_1,energy_source_2_transport_2,energy_source_2_transport_3,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,energy_storage_capacity_mwh,ferc_qualifying_facility,fluidized_bed_tech,fuel_type_code_pudl,fuel_type_count,latitude,longitude,minimum_load_mw,multiple_fuels,nameplate_power_factor,net_capacity_mwdc,operating_date,operating_switch,operational_status,operational_status_code,original_planned_operating_date,other_combustion_tech,other_modifications_date,other_planned_modifications,owned_by_non_utility,ownership_code,planned_derate_date,planned_energy_source_code_1,planned_modifications,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_derate_mw,planned_net_winter_capacity_uprate_mw,planned_new_capacity_mw,planned_new_prime_mover_code,planned_repower_date,planned_retirement_date,planned_uprate_date,previously_canceled,prime_mover_code,pulverized_coal_tech,reactive_power_output_mvar,retirement_date,rto_iso_lmp_node_id,rto_iso_location_wholesale_reporting_id,solid_fuel_gasification,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,state,stoker_tech,street_address,subcritical_tech,summer_capacity_estimate,summer_capacity_mw,summer_estimated_capability_mw,supercritical_tech,switch_oil_gas,syncronized_transmission_grid,technology_description,time_cold_shutdown_full_load_code,timezone,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,unit_id_pudl,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,geometry,index_right
706,2022-01-01,65285,24033,Prince George's County,"[24017, 24009, 51510, 51059, 11001, 24003, 240...",1811,Chalk Point Steam,12653,1869,Mirant Mid-Atlantic LLC,ST2,False,<NA>,False,364.0,<NA>,Aquasco,<NA>,Prince Georges,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BIT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,coal,2,38.544400,-76.686100,NaN,<NA>,NaN,NaN,1965-03-01,<NA>,retired,RE,NaT,<NA>,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,ST,True,NaN,2021-06-01,50816,chalkpt 20 kv chlkg2,False,<NA>,<NA>,<NA>,<NA>,MD,<NA>,P.O. 10,<NA>,<NA>,336.9,NaN,True,<NA>,<NA>,Conventional Steam Coal,<NA>,America/New_York,X,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,344.5,NaN,20608,POINT (-76.68610 38.54440),749
27798,2022-01-01,1771,26041,Delta County,"[26089, 26153, 55029, 26109, 26103, 26003]",1849,Escanaba,58246,1658,Escanaba Operating Services LLC,2,False,<NA>,False,11.5,<NA>,Escanaba,<NA>,Delta,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BIT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,coal,2,45.771667,-87.063920,NaN,<NA>,NaN,NaN,1958-05-01,<NA>,retired,RE,NaT,<NA>,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,ST,<NA>,NaN,2016-01-01,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,MI,True,2000 Power Plant Rd,True,<NA>,13.0,NaN,<NA>,<NA>,<NA>,Conventional Steam Coal,<NA>,America/Detroit,X,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,13.0,NaN,49829,POINT (-87.06392 45.77167),1174
27800,2022-01-01,1769,26103,Marquette County,"[26109, 26041, 26043, 26003, 26071, 26013, 260...",466,Presque Isle,20847,363,Wisconsin Electric Power Co,9,False,<NA>,False,90.0,<NA>,Marquette,<NA>,Marquette,NaT,monthly_update,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,SUB,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,coal,1,4

In [29]:
eia_county_geoms = create_geometries_df(eia_county_df, census_geometry="county")

In [ ]:
make_matplotlib_map(eia_county_geoms)

In [ ]:
make_matplotlib_map(eia_county_geoms, only_conus=True)

# Combine coal mine and generator closure data to see all qualifying areas

In [30]:
# tract
full_tract_df = pd.concat(
    [msha_df[["tract_id_fips", "adjacent_id_fips"]],
     eia_df[["tract_id_fips", "adjacent_id_fips"]]])
n_primary_geoms = len(full_tract_df["tract_id_fips"].drop_duplicates())
n_all_geoms = len(
        pd.concat([full_tract_df["tract_id_fips"], full_tract_df.adjacent_id_fips.explode()]).drop_duplicates())
print(f"Number of tracts with closed mines or generators: {n_primary_geoms}")
print(f"Number of qualifying Census tracts (including adjacent tracts): {n_all_geoms}")

Number of tracts with closed mines or generators: 1254
Number of qualifying Census tracts (including adjacent tracts): 5042


In [31]:
full_tract_df

,tract_id_fips,adjacent_id_fips
63,01043965700,"[01127021900, 01009050601, 01127020800, 010439..."
86,01073011600,"[01073012001, 01073012103, 01073012104, 010730..."
91,01073012104,"[01073012302, 01073012403, 01073012200, 010730..."
97,01127021400,"[01127021600, 01127021300, 01127021500, 011270..."
116,01073011400,"[01073011600, 01073011500, 01073011704, 011270..."
...,...,...
297235,17137951400,"[17171970600, 17137951500, 17017960200, 170179..."
297236,17137951400,"[17171970600, 17137951500, 17017960200, 170179..."
297239,17033880100,"[17033880500, 17033880200, 17033880300, 170230..."
286092,27009020206,"[27009021101, 27009020300, 27145010102, 271450..."


### Qualifying coal community Census tracts

In [ ]:
make_matplotlib_map(create_geometries_df(full_tract_df, census_geometry="tract"))

In [ ]:
# county
full_county_df = pd.concat(
    [msha_county_df[["county_id_fips", "adjacent_id_fips"]],
     eia_county_df[["county_id_fips", "adjacent_id_fips"]]])
n_primary_geoms = len(full_county_df["county_id_fips"].drop_duplicates())
n_all_geoms = len(
        pd.concat([full_county_df["county_id_fips"], full_county_df.adjacent_id_fips.explode()]).drop_duplicates())
print(f"Number of counties with closed mines or generators: {n_primary_geoms}")
print(f"Number of qualifying Census counties (including adjacent counties): {n_all_geoms}")

### Qualifying coal community counties

In [ ]:
make_matplotlib_map(create_geometries_df(full_county_df, census_geometry="county"))

### Qualifying coal communities broken down by generator and mine closures

In [ ]:
from energy_comms.output.basic_viz import combine_gen_and_mine_geoms

Start with tract

In [ ]:
full_tract_geoms = combine_gen_and_mine_geoms(msha_tract_geoms, eia_tract_geoms)

In [ ]:
make_matplotlib_map(full_tract_geoms)

In [ ]:
make_matplotlib_map(full_tract_geoms, only_conus=True)

By county

In [ ]:
full_county_geoms = combine_gen_and_mine_geoms(msha_county_geoms, eia_county_geoms)

In [ ]:
make_matplotlib_map(full_county_geoms)

In [ ]:
make_matplotlib_map(full_county_geoms, only_conus=True)

output for RMI Patio team:

In [161]:
def explode_adj_tracts(df,coal_type):
    new_df = df[['adjacent_id_fips']]
    #make a string
    new_df['adjacent_id_fips'] = new_df['adjacent_id_fips'].astype(str)
    #replace characters not in fips
    new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace('[','')
    new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace(']','')
    new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace("'",'')
    new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace(' ','')

    #make individual rows from list
    new_df = new_df.assign(adjacent_id_fips=new_df['adjacent_id_fips'].str.split(',')).explode('adjacent_id_fips')
    new_df = new_df.reset_index(drop=True)

    #rename column
    new_df = new_df.rename(columns={'adjacent_id_fips':'fips_tract'})

    #get county fips from tract column
    new_df['fips_county'] = new_df['fips_tract'].str[:5]

    # fill in patio columns
    new_df['qualifying_area'] = 'census_tract'

    new_df['criteria'] = 'coal_community'

    new_df['area_title'] = 'adjacent_' + coal_type

    # drop duplicates since there might be duplicate adjacent tracts

    new_df = new_df.drop_duplicates(subset='fips_tract')

    return new_df
    

In [162]:
msha_adjacent = explode_adj_tracts(msha_df,'mine')

/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['adjacent_id_fips'] = new_df['adjacent_id_fips'].astype(str)
/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace('[','')
/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

msha mines

In [164]:
#get columns we want
msha_df = msha_df[['current_mine_name','tract_id_fips']]
# rename to fit
msha_df = msha_df.rename(columns={'current_mine_name':'area_title','tract_id_fips':'fips_tract'})
# make county column
msha_df['fips_county'] = msha_df['fips_tract'].str[0:5]

#patio columns
msha_df['qualifying_area'] = 'census_tract'
msha_df['criteria'] = 'coal_community'

eia

adjacent gen

In [166]:
eia_adjacent = explode_adj_tracts(eia_df,'gen')

/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['adjacent_id_fips'] = new_df['adjacent_id_fips'].astype(str)
/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_df['adjacent_id_fips']= new_df['adjacent_id_fips'].str.replace('[','')
/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_2320/733128013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

gen

In [170]:
eia_df = eia_df[['plant_name_eia','tract_id_fips']]

eia_df = eia_df.rename(columns={'plant_name_eia':'area_title','tract_id_fips':'fips_tract'})
# make county column
eia_df['fips_county'] = eia_df['fips_tract'].str[0:5]

#patio columns
eia_df['qualifying_area'] = 'census_tract'
eia_df['criteria'] = 'coal_community'

concat everything

In [171]:
patio_mine = pd.concat([msha_df,msha_adjacent])
patio_gens = pd.concat([eia_df,eia_adjacent])

patio_coal = pd.concat([patio_mine,patio_gens])
patio_coal

,area_title,fips_tract,fips_county,qualifying_area,criteria
63,Arkadelphia,01043965700,01043,census_tract,coal_community
86,Chetopa,01073011600,01073,census_tract,coal_community
91,Bessie Mine,01073012104,01073,census_tract,coal_community
97,Gorgas #7 Preparation Plant,01127021400,01127,census_tract,coal_community
116,Beltona Mine,01073011400,01073,census_tract,coal_community
...,...,...,...,...,...
5568,adjacent_gen,42079215600,42079,census_tract,coal_community
5569,adjacent_gen,42079215702,42079,census_tract,coal_community
5570,adjacent_gen,42079213900,42079,census_tract,coal_community
5571,adjacent_gen,42079215800,42079,census_tract,coal_community
